<a href="https://colab.research.google.com/github/BhardwajArjit/Research-Paper-Replication/blob/main/CaaM_Replication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook replicates the research paper titled "Causal Attention for Unbiased Visual Recognition" with PyTorch.

The link to the paper: https://arxiv.org/abs/2108.08782